In [ ]:
#Import needed libraries
import timeit
tic=timeit.default_timer()
import re
import os
import sys
import random
import pickle
import implicit
import numpy as np
import pandas as pd
import snowflake.connector
import pandas.io.sql as sql
import scipy.sparse as sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import MinMaxScaler
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

#Connect to Snowflake database and load data
cnn = snowflake.connector.connect(
                user='bagabi',
                password='Ben@2020',
                account='sears_hs_prod.us-east-1'
                )
cursor = cnn.cursor()

generic = """SELECT 
 a.PRT_ORD_DT,
 a.PRT_ORD_LN_NO,
 a.PRT_DIV_NO,
 a.PRT_PRC_LIS_SRC_NO,
 a.PRT_NO,
 a.div_pls,
 a.item_id, 
 b.ATTRIBUTEID,  
 b.PARTID
 FROM
(
SELECT PRT_ORD_DT,
 PRT_ORD_LN_NO,
 PRT_DIV_NO,
 PRT_PRC_LIS_SRC_NO,
 PRT_NO, 
 LPAD(prt_div_no, 4, '0') || LPAD(prt_prc_lis_src_no, 3, '0') AS div_pls,
 LPAD(prt_div_no, 4, '0') || LPAD(prt_prc_lis_src_no, 3, '0') || prt_no AS item_id
FROM
"PRD_PARTSDIRECT"."BATCH"."PRTDTPL_PRT_LN"
) a
 left join "PRD_KM"."KM"."PART_ATTRIBUTE" b
 ON b.PARTID=a.item_id
 WHERE b.ATTRIBUTEID=138 
 AND b.ACTIVE = 1
 AND a.prt_ord_dt >= '2019-01-01' 
"""
query_generic = pd.read_sql_query(generic, cnn)

#Generic parts catalog recommendations
data = query_generic[['DIV_PLS','ITEM_ID','PRT_ORD_LN_NO']]
data = data.groupby(['DIV_PLS','ITEM_ID']).agg(
    Count=pd.NamedAgg(column='PRT_ORD_LN_NO', aggfunc='nunique')   
)
data.reset_index(drop=False, inplace=True)

user_category = CategoricalDtype(sorted(data.DIV_PLS.unique()), ordered=True)
item_category = CategoricalDtype(sorted(data.ITEM_ID.unique()), ordered=True)

row = data['DIV_PLS'].astype(user_category).cat.codes
col = data['ITEM_ID'].astype(item_category).cat.codes

#Transform two column raw data frame into item-item sparse matrix
sparse_matrix = csr_matrix((data['Count'], (row, col)),shape=(user_category.categories.size,item_category.categories.size))
sparse_df = pd.SparseDataFrame(sparse_matrix,index=user_category.categories,columns=item_category.categories,default_fill_value=0)

#Using sparse matrix create co-occurrence matrix
co_matrix = sparse_matrix.transpose().dot(sparse_matrix)
co_matrix.setdiag(0)
co_df = pd.SparseDataFrame(co_matrix,
                               index=item_category.categories,
                               columns=item_category.categories,
                               default_fill_value=0)

#Filter out nonzero index in co-occurrence matrix
idx = pd.np.nonzero(co_matrix)

#Create subjects and peers list
rows = idx[0]
columns = idx[1]
subjects = [item_category.categories[i] for i in rows]
peers = [item_category.categories[i] for i in columns]

#Extract the co-occurrence counts for pair of parts
occur = co_matrix[idx].tolist()[0]

#Create final recommended dataframe
df_recommend = pd.DataFrame.from_records(zip(subjects, peers, occur),
                                         columns = ['PARTID', 'RECOMMENDED_PARTID', 'PCT_SCORE'])
result = df_recommend.groupby(['PARTID','RECOMMENDED_PARTID']).agg({'PCT_SCORE': 'sum'})
result = result.groupby(level=['PARTID']).apply(lambda x: 100 * x / float(x.sum()))
result.reset_index(drop=False,inplace=True)
result['PCT_SCORE'] = round(result['PCT_SCORE']/1,1)
finalResults = result.sort_values(by=['PARTID','PCT_SCORE'], ascending=[False,False])

finalResults = finalResults[(finalResults['PARTID'] != finalResults['PARTID'].shift()) | (finalResults['RECOMMENDED_PARTID'] \
                                                                         != finalResults['RECOMMENDED_PARTID'].shift())]

finalResults['PARTID'] = finalResults['PARTID'].astype('str')
finalResults['RECOMMENDED_PARTID'] = finalResults['RECOMMENDED_PARTID'].astype('str')                                         
#finalResults.to_csv('PART_RECOMMENDATION.csv', index=False)

toc=timeit.default_timer()
Elapsed = toc - tic 
hours, rem = divmod(Elapsed, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))